In [21]:
import requests
import pandas as pd
import re

url = "https://raw.githubusercontent.com/Lorenzo1208/Projet_TrouveTonJob/main/data.json"
response = requests.get(url)
data = response.json()
df = pd.DataFrame(data)

# Supprimer les caractères de nouvelle ligne et formater les données
df['Intitulé du poste'] = df['Intitulé du poste'].apply(lambda x: [i.strip() for i in x])
df['Date de publication'] = df['Date de publication'].apply(lambda x: x.strip().replace("postée ", "").replace("il y a ", ""))
df['lieu'] = df['lieu'].apply(lambda x: [i.strip() for i in x])
df['competences'] = df['competences'].apply(lambda x: [i.strip() for i in x])
df['Type de poste'] = df['Type de poste'].apply(lambda x: [i.strip() for i in x])

# Extraction de la valeur numérique de la colonne lieu et stockage dans la colonne 'salaire'.
df['salaire'] = df['lieu'].apply(lambda x: [i for i in x if i.find("€")!=-1])
df['salaire'] = df['salaire'].apply(lambda x: x[0].replace('\n','') if x else '')
df['lieu'] = df['lieu'].apply(lambda x: x[0].replace('\d','') if x else '')

df_clean = df
df_clean.head(10)

,Intitulé du poste,Date de publication,lieu,competences,Type de poste,salaire
0,[Data Analyst - Publicité],21 jours,Paris,"[sql, t-sql, outils]","[, , Sept Lieues SAS, , Cabinet de recrutement...",
1,[Data analyst - H/F],1 mois,"Paris, La Défense","[support, outils, agile, design, big data]","[, , EDF France, , Client final, , , CDI - tem...",
2,[Customer Data Analyst (H/F)],27 jours,Paris,"[crm, data quality, sql, sas, python]","[, , Hermès, , Client final, , , CDI - temps p...",
3,[Master Data Analyst H/F],21 jours,Moussy-le-Vieux,"[si, support, sap]","[, , TEREOS, , Client final, , , CDI - temps p...",
4,[Portfolio Data Analyst H/F],3 jours,France,"[outils, support, decisionnel, sql, test]","[, , Safran-Group, , Client final, , , CDI - t...",
5,[Data Analyst BO/Qlick Sense (h/f)],15 jours,Maisons Alfort,"[support, sql, data management, business objec...","[, , FED IT, , Cabinet de recrutement, , , CDD...","45.000,00 € - 50.000,00 € / an"
6,[Product Data Analyst - BI - H/F],28 jours,Paris,"[cloud, paas, qlik, iaas, support]","[, , Cegid, , Client final, , , CDI - temps pl...",
7,[Data Analyst / Chef de Projet BI (h/f)],1 mois,PARIS,"[data management, business intelligence, bases...","[, , FED IT, , Cabinet de recrutement, , , CDI...","60.000,00 € - 62.000,00 € / an"
8,[Business analyst senior DATA CRM - Paris H/F],12 heures,Paris,"[crm, agile, decisionnel, réseaux, réseau]","[, , Credit Agricole, , Client final, , , CDI ...",
9,[Data scientist / Analyst and software develop...,13 jours,Rueil-Malmaison,"[big data, scrum, intelligence artificielle, e...","[, , Vinci, , Client final, , , Stage]",
